In [11]:
import pandas as pd
import seaborn as sns 
import matplotlib.pyplot as plt 
import numpy as np 
import math
import geopandas as gpd
import csv

#import geoplot as gplt


from datetime import datetime
from pylab import *
from shapely.ops import nearest_points

In [12]:
df = pd.read_csv('clean_nyc_airbnb.csv')
df2 = pd.read_csv('clean_nyc_sale.csv')

In [13]:
df

,id,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,activity,occupancy_%,how_far_km,how_far_km.1
0,2595,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,Active,3.0,7.330251,7.330251
1,3831,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-05-07,4.64,1,194,Active,47.0,9.695046,9.695046
2,5099,Manhattan,Murray Hill,40.74767,-73.97500,Entire home/apt,200,3,74,2019-06-22,0.59,1,129,Active,65.0,7.263750,7.263750
3,5178,Manhattan,Hell's Kitchen,40.76489,-73.98493,Private room,79,2,430,2019-06-24,3.47,1,220,Active,40.0,7.695761,7.695761
4,5238,Manhattan,Chinatown,40.71344,-73.99037,Entire home/apt,150,1,160,2019-09-06,1.33,4,188,Active,48.0,7.823571,7.823571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25983,36425863,Manhattan,Upper East Side,40.78099,-73.95366,Private room,129,1,1,2019-07-07,1.00,1,147,Active,60.0,8.613975,8.613975
25984,36427429,Queens,Flushing,40.75104,-73.81459,Private room,45,1,1,2019-07-07,1.00,6,339,Active,7.0,15.835060,15.835060
25985,36438336,Staten Island,Great Kills,40.54179,-74.14275,Private room,235,1,1,2019-07-07,1.00,1,87,Active,76.0,26.005483,26.005483
25986,36442252,Bronx,Mott Haven,40.80787,-73.92400,Entire home/apt,100,1,2,2019-07-07,2.00,1,40,Active,89.0,11.133406,11.133406


In [14]:
df2

,neighbourhood_group,neighbourhood,sale_price,sale_date,latitude,longitude,how_far_km,how_far_km.1
0,Manhattan,CHELSEA,5200000,2018-12-06,40.742107,-73.998510,7.454557,7.454557
1,Manhattan,CHELSEA,540000,2018-12-06,40.745466,-73.998917,7.501481,7.501481
2,Manhattan,CHINATOWN,0,2018-12-06,40.717094,-73.996652,7.737963,7.737963
3,Manhattan,CLINTON,336648,2018-12-06,40.768225,-73.986419,7.868896,7.868896
4,Manhattan,CLINTON,18400000,2018-12-06,40.760653,-73.991914,7.676230,7.676230
...,...,...,...,...,...,...,...,...
77747,Staten Island,WOODROW,0,2019-01-25,40.539191,-74.222818,30.246898,30.246898
77748,Staten Island,WOODROW,590000,2019-01-11,40.536236,-74.209270,29.721766,29.721766
77749,Staten Island,WOODROW,0,2019-01-31,40.535069,-74.215249,30.141944,30.141944
77750,Staten Island,WOODROW,0,2019-01-24,40.539712,-74.215793,29.808690,29.808690


In [15]:
#calculate baseline 

years = 15 

#med_occ = df_active['occupancy_%'].median()
med_sale = df2['sale_price'].median()

expected_annual_return = med_sale / 15
expected_roi = expected_annual_return / med_sale

print(f'optimal occupancy: {med_occ}%')
print(f'median sale price: ${med_sale}')
print(f'pay 15-year-period: ${expected_annual_return}')
print(f'optimal roi for 15-year-period: {expected_roi}')

NameError: name 'med_occ' is not defined

In [ ]:
#split into neighbourhood_group

br_sale = df2.loc[df2["neighbourhood_group"] == 'Bronx']
bk_sale = df2.loc[df2["neighbourhood_group"] == 'Brooklyn']
mn_sale = df2.loc[df2["neighbourhood_group"] == 'Manhattan']
qn_sale = df2.loc[df2["neighbourhood_group"] == 'Queens']
si_sale = df2.loc[df2["neighbourhood_group"] == 'Staten Island']

br_airbnb = df_active.loc[df_active["neighbourhood_group"] == 'Bronx']
bk_airbnb = df_active.loc[df_active["neighbourhood_group"] == 'Brooklyn']
mn_airbnb = df_active.loc[df_active["neighbourhood_group"] == 'Manhattan']
qn_airbnb = df_active.loc[df_active["neighbourhood_group"] == 'Queens']
si_airbnb = df_active.loc[df_active["neighbourhood_group"] == 'Staten Island']

In [ ]:
#divide into types of houses
shared_room = df.loc[df['room_type'] == 'Shared room']
apt = df.loc[df['room_type'] == 'Entire home/apt']
private_room = df.loc[df['room_type'] == 'Private room']

In [ ]:
# plot location in relation to Neighbourhood group 
title = 'Neighbourhood Group Location'
plt.figure(figsize=(10,6))
sns.scatterplot(data.longitude,data.latitude,hue=data.neighbourhood_group).set_title(title)
plt.ioff()

In [ ]:
sns.set(style="white", palette="muted", color_codes=True)

title = ""
sns.catplot(x='neighbourhood_group', kind="count", hue="room_type", data=df)
